<a href="https://colab.research.google.com/github/DevelopBG/transformer/blob/main/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, DatasetDict,Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [3]:
# the standford sentiment treebank. the task to predict the sentiment of a giver sentence
dataset = load_dataset("glue", "sst2")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [4]:
model_checkpoint = 'roberta-base'

# define label maps
id_label = {0:"negative",1:"positive"}
label_id = {"negative":0, "positive":1}
# generate classificaition model form model checkpoint

Robertamodel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id_label,
    label2id=label_id
)

Robertamodel

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [5]:
# creating tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
# add pad token if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  Robertamodel.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
# create tokenizer function
def tokenize_function(examples):
  # extract text
  text = examples['sentence']
  #tokenize and truncate text
  tokenizer.truncatioin_side='left'
  tokenized_inputs = tokenizer(
      text,return_tensors = 'np',
      truncation = True,
      max_length = 512
  )
  return tokenized_inputs

In [7]:
# tokenize training and validation datasets
tokenized_dataset= dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [8]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer= tokenizer)


In [9]:
accuracy = evaluate.load('accuracy')

# define an evaluation function to pass into trainer later

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)
  return {"accuracy":accuracy.compute(predictions=predictions, references = labels)}


In [10]:
# lisst of examples
text_list = ["The movie was a breathtaking masterpiece that kept me hooked from start to finish.",
             "The plot was predictable and painfully slow.",
             "I regret wasting two hours on this boring film.",
             "The performances were outstanding, especially the lead actor’s emotional depth.",
             "The visuals and soundtrack made the film an unforgettable experience."]
print("Untrained model predictions")
print("-------------------")
for text in text_list:
  inputs = tokenizer.encode(text, return_tensors='pt')
  logits = Robertamodel(inputs).logits
  predictions = torch.argmax(logits)
  print(text + "-" + id_label[predictions.tolist()])

Untrained model predictions
-------------------
The movie was a breathtaking masterpiece that kept me hooked from start to finish.-negative
The plot was predictable and painfully slow.-negative
I regret wasting two hours on this boring film.-negative
The performances were outstanding, especially the lead actor’s emotional depth.-negative
The visuals and soundtrack made the film an unforgettable experience.-negative


In [25]:
# Robertamodel.unload()
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r = 1,
                         lora_alpha = 32,
                         lora_dropout = 0.01,
                         target_modules = ['query'])

In [26]:
model = get_peft_model(Robertamodel, peft_config)
model.print_trainable_parameters()

trainable params: 610,562 || all params: 125,257,732 || trainable%: 0.4874


In [27]:
lr = 1e-3
batch_size = 16
num_epochs = 1

In [28]:
training_args = TrainingArguments(
    output_dir = model_checkpoint+'-lora-text-classification',
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    # save_strategy = 'epoch',
    # evaluation_strategy = 'epoch',
    # load_best_model_at_end = True
    # # load_best_model_at_end = True
    )

In [29]:
trainer = Trainer(
    model = Robertamodel,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

/tmp/ipython-input-2341022768.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bghosh to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.390700
1000,0.326600
1500,0.292100
2000,0.287900
2500,0.277400
3000,0.271700
3500,0.260000
4000,0.243900


TrainOutput(global_step=4210, training_loss=0.29227481035608577, metrics={'train_runtime': 358.679, 'train_samples_per_second': 187.77, 'train_steps_per_second': 11.738, 'total_flos': 1246125325382808.0, 'train_loss': 0.29227481035608577, 'epoch': 1.0})

In [31]:
# trained model prediction
Robertamodel.to("cpu")
for text in text_list:
  inputs = tokenizer.encode(text, return_tensors = 'pt').to('cpu')
  logits = Robertamodel(inputs).logits
  predictions = torch.argmax(logits)
  print(text + "-" + id_label[predictions.tolist()])

The movie was a breathtaking masterpiece that kept me hooked from start to finish.-positive
The plot was predictable and painfully slow.-negative
I regret wasting two hours on this boring film.-negative
The performances were outstanding, especially the lead actor’s emotional depth.-positive
The visuals and soundtrack made the film an unforgettable experience.-positive
